In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [3]:
p = 10
a = 10
epochs = 20_000
n_t = 6
n_s = 6
n_t_rf = 6
v_bb_np = np.random.randn(n_t_rf, n_s) + 1.j * np.random.randn(n_t_rf, n_s)
v_rf_np = np.random.randn(n_t_rf, n_s) + 1.j * np.random.randn(n_t_rf, n_s)

In [57]:
x = np.random.randn(n_s, 1) + 1.j * np.random.randn(n_s, 1)
len(x)

6

In [53]:
x_h = np.conjugate(x.T)
e = x @ x_h
x.shape, x_h.shape, e.shape
e

array([[ 5.58631744+0.j        ,  4.60471328-1.40024603j,
        -0.99232879+0.02801696j, -2.32741988+1.76431397j,
        -1.33578515-0.91966384j, -2.45722792+1.59497781j],
       [ 4.60471328+1.40024603j,  4.14657305+0.j        ,
        -0.82498359-0.22563959j, -2.360692  +0.87091354j,
        -0.87054702-1.09288744j, -2.42524553+0.69879521j],
       [-0.99232879-0.02801696j, -0.82498359+0.22563959j,
         0.17641342+0.j        ,  0.42228114-0.30173228j,
         0.23267061+0.17006437j,  0.44449039-0.27100113j],
       [-2.32741988-1.76431397j, -2.360692  -0.87091354j,
         0.42228114+0.30173228j,  1.52688908+0.j        ,
         0.26607102+0.8050363j ,  1.52748977+0.11154727j],
       [-1.33578515+0.91966384j, -0.87054702+1.09288744j,
         0.23267061-0.17006437j,  0.26607102-0.8050363j ,
         0.47081169+0.j        ,  0.32498782-0.78591512j],
       [-2.45722792-1.59497781j, -2.42524553-0.69879521j,
         0.44449039+0.27100113j,  1.52748977-0.11154727j,
         

In [25]:
v_bb_np.flatten().imag

array([-1.03706383,  2.25253468,  0.8986551 ,  0.645868  , -1.10503888,
       -1.44029431, -1.00544302, -0.19932236, -2.08376617, -0.14306814,
       -0.69612858, -0.23744182,  0.6491041 , -1.53246247,  0.01000787,
        0.26616946, -0.7214835 ,  1.14361478,  0.70676741,  0.15469973,
       -0.72763227,  0.99712208, -0.65931373, -0.5805754 , -0.71762356,
       -1.82294461, -0.79339756,  0.31705508,  1.58367802,  0.11699612,
        0.19466036, -0.48870666, -0.55796901,  0.54107478, -0.18457285,
       -0.37129372])

In [26]:
def flat_real_imag(arr: np.ndarray):
    f = arr.flatten()
    return f.real, f.imag

In [31]:
def get_state(v_bb: np.ndarray, w_rf: np.ndarray):
    return np.concatenate([*flat_real_imag(v_bb), *flat_real_imag(v_bb)])

In [32]:
get_state(v_bb, v_bb)

array([ 0.4460873 ,  1.5255541 , -1.1319971 , -0.18493727, -0.12071718,
       -2.2647111 ,  1.0497689 , -0.17096573, -0.9410859 , -2.2242413 ,
       -1.3178209 , -0.0877995 ,  0.29217592, -2.1763968 , -0.2229299 ,
       -0.13722308,  0.34303504,  0.35447958,  0.8836703 , -2.2909994 ,
       -1.7520922 , -0.77110964,  0.69725233, -0.68626   ,  0.64855576,
       -0.23175158,  0.877162  , -0.48805836,  0.17112899, -0.37518507,
        1.2338338 , -1.567395  ,  0.38419798, -0.43518013, -1.5562775 ,
       -0.8371242 , -1.0370638 ,  2.2525346 ,  0.8986551 ,  0.645868  ,
       -1.1050389 , -1.4402943 , -1.005443  , -0.19932236, -2.0837662 ,
       -0.14306813, -0.6961286 , -0.23744182,  0.6491041 , -1.5324625 ,
        0.01000787,  0.26616946, -0.7214835 ,  1.1436148 ,  0.70676744,
        0.15469973, -0.7276323 ,  0.9971221 , -0.65931374, -0.5805754 ,
       -0.71762353, -1.8229446 , -0.79339755,  0.31705508,  1.583678  ,
        0.11699612,  0.19466037, -0.48870668, -0.55796903,  0.54

In [5]:
v_bb = torch.tensor(v_bb_np, dtype=torch.cfloat)

In [9]:
(v_bb / v_bb.abs()).abs()

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])

In [91]:
def h(x: torch.tensor) -> torch.tensor:
    return torch.conj(x.transpose(1, 0))

def constraint(v_rf: torch.tensor, v_bb: torch.tensor) -> torch.tensor:
    t1 = torch.matmul(v_rf, v_bb)
    t2 = torch.matmul(t1, h(v_bb))
    t3 = torch.matmul(t2, h(v_rf))
    tr = torch.trace(t3)
    return tr

def find_v_rf(v_rf: np.ndarray, v_bb: np.ndarray, p: float, a: float, epochs: int):
    p -= p / a
    v_bb = torch.tensor(v_bb, dtype=torch.cfloat)
    v_rf = torch.tensor(v_rf, dtype=torch.cfloat).requires_grad_()
    losses = []
    optimizer = torch.optim.Adam([v_rf])
    for i in range(epochs):
        optimizer.zero_grad()
        loss = ((constraint(v_rf, v_bb) - p) ** 2).mean()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    del optimizer
    return v_rf.detach(), v_bb, losses

def plot_loss(v_rf, v_bb, err, start=0):
    y = err[-start:]
    l = len(err) - start
    x = list(range(len(y)))
    c = constraint(v_rf, v_bb).item()
    title = f"LOSS | $ Constraint = {round(c.real, 4)} + j{round(c.imag, 4)} $ | last {l}"
    fig, ax = plt.subplots(dpi=200);
    ax.plot(x, np.real(y), label="$ Real(loss) $");
    ax.plot(x, np.imag(y), label="$ Imag(loss) $");
    ax.set_title(title);
    ax.grid();
    ax.legend();
    return fig